## Notebook for spike sorting from .kwd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import socket
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from datetime import timedelta
from importlib import reload

# pipeline imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from pipefinch.h5tools.kwik import kutil
from pipefinch.pipeline import probes


from pipefinch.h5tools.kwik import kwdfunctions as kwdf

from intan2kwik import kwd

#mountainsort imports (for sorting)
#import mountainlab_pytools.mlproc as mlp

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')
logger.info('Hostname {}'.format(socket.gethostname()))

2019-02-27 15:32:11,289 - root - INFO - Logger set
2019-02-27 15:32:11,290 - root - INFO - Hostname zinch


### Session parameters and raw files

In [5]:
from pipefinch.pipeline import filestructure as et
reload(et)
reload(kwd)

sess_par = {'bird': 'b11j11',
           'sess': '2019-02-26_2250_01',
           'probe': 'probe_0' # probe to sort ('probe_0', 'probe_1') (to lookup in the rig_par which port to extract)
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], live_like_animals=True)

# mountainsort parameters
sort_params = {'adjacency_radius': -1,
              'detect_threshold': 2,
              'freq_min': 600}

# differetn mountainsort parameters
ds_params = {'detect_sign': -1}


# convenient paths
kwik_folder = exp_struct['folders']['kwik']
msort_folder = exp_struct['folders']['msort']
raw_folder = exp_struct['folders']['raw']
kwd_path = exp_struct['files']['kwd']
bin_path = exp_struct['files']['mda_raw']

In [6]:
exp_struct

{'folders': {'raw': '/mnt/zuperfinchjr/Data/b11j11/Ephys/raw/2019-02-26_2250_01',
  'kwik': '/media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01',
  'msort': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01'},
 'files': {'par': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/params.json',
  'set': '/mnt/zuperfinchjr/Data/b11j11/Ephys/raw/2019-02-26_2250_01/settings.isf',
  'rig': '/mnt/zuperfinchjr/Data/b11j11/Ephys/raw/2019-02-26_2250_01/rig.json',
  'kwd': '/media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01/streams.kwd',
  'kwik': '/media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01/spikes.kwik',
  'kwe': '/media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01/events.kwe',
  'mda_raw': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/raw.mda'}}

### convert the whole session to a .kwd file
Conversion sends every .rhd file in the folder to a rec in the .kwd file (experiment.kwd in the session ss folder)
All of the files and all of the channels are converted; filtering and subselection of sub-epochs and channels occurs later.
The .kwd is raw data, only in a friendlier format.


#### Make a file for the session for the first time

In [7]:
reload(kwd)
## Convert the whole session to a kwd file
os.makedirs(kwik_folder, exist_ok=True)
first_intan_hdr, sess_pd = kwd.intan_to_kwd(raw_folder, kwd_path)

2019-02-27 15:33:12,412 - intan2kwik.kwd - INFO - reading intan chans data across all of rec /mnt/zuperfinchjr/Data/b11j11/Ephys/raw/2019-02-26_2250_01
2019-02-27 15:33:12,882 - intan2kwik.kwd - INFO - Found 50 .rhd files split in 40 recordings
2019-02-27 15:33:12,883 - intan2kwik.kwd - INFO - dest file: /media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01/streams.kwd


2019-02-27 15:40:40,596 - intan2kwik.kwd - INFO - moving back to /media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01/streams.kwd


2019-02-27 15:40:51,894 - intan2kwik.kwd - INFO - removing temp file


#### Update a session with subsequently recorded rhd files

In [4]:
reload(kwdf)
_, nu_pd, _ = kwdf.update_kwd(kwd_path, raw_folder)

2019-02-27 10:38:38,793 - pipefinch.h5tools.kwik.kwdfunctions - INFO - updating kwd file /media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-25_2250_01/streams.kwd from folder /mnt/zuperfinchjr/Data/b11j11/Ephys/raw/2019-02-25_2250_01
2019-02-27 10:38:38,893 - pipefinch.h5tools.kwik.kwdfunctions - INFO - No new files to add to the file


## Make .mda file with a set of recordings in a session
 - pick all in port A
 - get all rec within a time range


In [8]:
# get the session meta
pd_meta = kwdf.get_all_rec_meta(kwd_path)
pd_meta.head()


,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,dig_channel_names,is_multiSampleRate_data,valid_samples,samples_count
0,16,0,20000.0,0,2019-02-26 08:13:13,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1...",1199520
1,16,1,20000.0,0,2019-02-26 08:13:43,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[4991040.0, 4991040.0, 4991040.0, 4991040.0, 4...",4991040
2,16,2,20000.0,0,2019-02-26 08:26:27,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1...",1199520
3,16,3,20000.0,0,2019-02-26 08:26:57,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[448560.0, 448560.0, 448560.0, 448560.0, 44856...",448560
4,16,4,20000.0,0,2019-02-26 09:16:17,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1...",1199520


### pick a time interval of the recordings

In [84]:
def select_time_span(meta_pd, start, span_minutes):
    end = start + timedelta(minutes=span_minutes)
    pd_selection = meta_pd.loc[meta_pd['start_time'].between(start, end)]
    return pd_selection

pd_meta_selection = select_time_span(pd_meta, pd_meta['start_time'][0], 240)

In [85]:
# for instance
pd_meta_selection.head()

,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,dig_channel_names,is_multiSampleRate_data,valid_samples,samples_count
0,16,0,20000.0,0,2019-02-26 08:13:13,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1...",1199520
1,16,1,20000.0,0,2019-02-26 08:13:43,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[4991040.0, 4991040.0, 4991040.0, 4991040.0, 4...",4991040
2,16,2,20000.0,0,2019-02-26 08:26:27,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1...",1199520
3,16,3,20000.0,0,2019-02-26 08:26:57,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[448560.0, 448560.0, 448560.0, 448560.0, 44856...",448560
4,16,4,20000.0,0,2019-02-26 09:16:17,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[D-004, D-006, D-007, D-008, D-012, D-013, D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1...",1199520


In [86]:
# load the rig parameters
rig_par_file = exp_struct['files']['rig']
with open(rig_par_file, 'r') as fp:
    rig_par = json.load(fp)

# get the probe and the port where the probe was connected
selected_probe = sess_par['probe']
probe_port = rig_par['chan']['port'][selected_probe].strip('-')

# get the channel indices of the probe's port
wanted_chans = np.array([probe_port + '-'])  # all ephys channels

chan_list = kwdf.get_all_chan_names(pd_meta_selection, chan_filt=wanted_chans)

#all_rec_list = kutil.get_rec_list(exp_struct['files']['kwd'])
selection_rec_list = np.unique(pd_meta_selection['name'])

rec_chans = pd_meta.loc[pd_meta['name'] == selection_rec_list[0], 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)

# make the mda binary file
bin_path = exp_struct['files']['mda_raw']
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)
bin_file = kwdf.kwd_to_binary(exp_struct['files']['kwd'],
                              exp_struct['files']['mda_raw'],
                              chan_list=chan_list,
                              rec_list=selection_rec_list, header='mda')


2019-02-28 11:22:10,589 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Writing kwd_file /media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01/streams.kwd to binary
2019-02-28 11:22:10,643 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Channels to extract: ['D-004' 'D-006' 'D-007' 'D-008' 'D-012' 'D-013' 'D-014' 'D-017' 'D-019'
 'D-020' 'D-021' 'D-022' 'D-023' 'D-025' 'D-027' 'D-028' 'D-029' 'D-030'
 'D-031']
2019-02-28 11:22:10,644 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Will go through recs [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
2019-02-28 11:22:10,645 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Creating binary file /media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/raw.mda


2019-02-28 11:22:26,761 - pipefinch.h5tools.kwik.kwdfunctions - INFO - 850554000 elements written


## Scripts for sorting

### prep the files with their nice formats, locations and names


In [87]:
reload(msc)
file_paths, out_folder = msc.make_paths(exp_struct['folders']['msort'])
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)

# make the probe file
rec_chans = pd_meta.loc[pd_meta['name']==0, 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)
probe = rig_par['probe'][selected_probe]['model']
headstage = rig_par['probe'][selected_probe]['headstage']
probe_chans = rec_chans_idx - np.min(rec_chans_idx)
# try to make a probe. If it is not possible, force adjacency_radius to -1.
try:
    probe_geom = probes.make_map(probe, probe_chans)
    np.savetxt(file_paths['geom'], probe_geom, delimiter=',')
except KeyError as err:
    logger.warning('Probe could not be made, probe or headstage not found {} in probes.py. Will sort with no geometry'.format(err))
    sort_params['adjacency_radius'] = -1

# parameters to pass to the msort scripts
ds_params.update({'samplerate': int(kwdf.get_sampling_rate(pd_meta, 0)), # required
            })


with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))


2019-02-28 11:22:26,779 - root - WARNING - Probe could not be made, probe or headstage not found 'carbon-fiber' in probes.py. Will sort with no geometry
2019-02-28 11:22:26,781 - root - INFO - Created session par files /media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/params.json


In [89]:
file_paths

{'mda': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/raw.mda',
 'params': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/params.json',
 'geom': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/geom.csv',
 'filt': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/filt.mda.prv',
 'pre': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/pre.mda.prv',
 'firings': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings.mda',
 'firings_curated': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings_curated.mda',
 'cluster_metrics': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/cluster_metrics_curated.json'}

In [90]:
# test mountainsort method for reading sorting parameters
msc.read_dataset_params(exp_struct['folders']['msort'])

{'detect_sign': -1, 'samplerate': 20000}

In [93]:
output_dir = os.path.join(exp_struct['folders']['msort'], 'sort_out');
msc.sort_dataset(file_paths=file_paths, **sort_params);

2019-02-28 11:22:26,828 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/filt.mda.prv


2019-02-28 11:22:37,502 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 83a775d2d3925a7eab758ca17a9a9499dea7a9b4
[ Checking outputs... ]
{"timeseries_out":"/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/filt.mda.prv"}
Processing ouput - /media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/filt.mda.prv
false
{"timeseries_out":"/media/zinch/Windows/experiment/msort-tmp/output_83a775d2d3925a7eab758ca17a9a9499dea7a9b4_timeseries_out.mda"}
[ Checking process cache ... ]
[ Process ephys.bandpass_filter already completed. ]
[ Creating output prv for timeseries_out ... ]
[ Done. ]
RUNNING: ml-run-process ephys.whiten --inputs timeseries:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/filt.mda.prv --parameters --outputs timeseries_out:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/pre.mda.prv


2019-02-28 11:22:46,509 - pipefinch.sort.mountain.comre - INFO - Sorting


...

RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:-1 detect_threshold:2 --outputs firings_out:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings.mda


2019-02-28 11:22:50,979 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: a715bf09e7930c04ee4a68ce096d41f4e66b92e2
[ Checking outputs... ]
{"firings_out":"/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings.mda"}
Processing ouput - /media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings.mda
false
{"firings_out":"/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings.mda"}
[ Checking process cache ... ]
[ Process ms4alg.sort already completed. ]
[ Done. ]
RUNNING: ml-run-process ms3.cluster_metrics --inputs firings:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings.mda timeseries:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/pre.mda.prv --parameters samplerate:20000 --outputs cluster_metrics_out:/media/zinch/Windows/experiment/msort-tmp/mountainlab/tmp_short_term/out

2019-02-28 11:23:04,559 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 6b304512a0d375b00bd3d2fbbf3c4c5f79104a4d
[ Checking outputs... ]
{"metrics_out":"/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/cluster_metrics.json"}
Processing ouput - /media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/cluster_metrics.json
false
{"metrics_out":"/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/cluster_metrics.json"}
[ Checking process cache ... ]
[ Process ms3.combine_cluster_metrics already completed. ]
[ Done. ]
RUNNING: ml-run-process ms4alg.create_label_map --inputs metrics:/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/cluster_metrics.json --parameters --outputs label_map_out:/media/zinch/Windows/experiment/msort-tmp/mountainlab/tmp_short_term/output_label_map_out_082a66e04c4aba907124743e692b881740e5a3d5.prv
[ Gett

In [95]:
file_paths

{'mda': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/raw.mda',
 'params': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/params.json',
 'geom': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/geom.csv',
 'filt': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/filt.mda.prv',
 'pre': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/pre.mda.prv',
 'firings': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings.mda',
 'firings_curated': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings_curated.mda',
 'cluster_metrics': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/cluster_metrics_curated.json'}

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json

# After manual curation
 - save the curated spikes in the sort_out/firings_curated.mda
 - come back to the notebook and run 

In [96]:
from pipefinch.h5tools.kwik import kwikfunctions as kwkf
reload(kwkf)
reload(et)
firings_to_save = 'firings_curated' # 'curated' or 'firings' for default_output


metrics_to_save = 'cluster_metrics_curated' if firings_to_save == 'firings_curated' else 'cluster_metrics'
kwkf.mda_to_kwik(exp_struct['files']['kwd'],
                 exp_struct['files']['kwik'],
                 file_paths[firings_to_save],
                file_paths[metrics_to_save])


2019-02-28 11:38:34,374 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Creating kwik file /media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01/spikes.kwik from firings /media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/sort_out/firings_curated.mda


In [97]:
### extract all unit waveforms
from pipefinch.neural import units
reload(units)
units.get_all_unit_waveforms(exp_struct['files']['kwik'], exp_struct['files']['kwd'], port=probe_port)

2019-02-28 11:38:35,407 - pipefinch.neural.units - INFO - About to get all waveforms for 40 units in file /media/zinch/Windows/experiment/b11j11/Ephys/kwik/2019-02-26_2250_01/spikes.kwik


0

In [83]:
def msort_cleanup(exp_struct: dict):
    # remove the mda files and try to cleanup the msort temp location
    mda_raw_path = exp_struct['files']['mda_raw']
    logger.info('removing intermediate msort mda file {}'.format(mda_raw_path))
    os.remove(mda_raw_path)

def msort_tmp_clean():
    tmp_dir = os.path.abspath(os.environ['ML_TEMPORARY_DIRECTORY'])
    logger.info('Cleaning up msort temp dir {}'.format(tmp_dir))
    
#msort_tmp_clean()
msort_cleanup(exp_struct)

2019-02-28 11:19:23,917 - root - INFO - removing intermediate msort mda file /media/zinch/Windows/experiment/b11j11/Ephys/msort/2019-02-26_2250_01/raw.mda


In [165]:
 exp_struct['files']

{'par': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-13_1750_01/params.json',
 'set': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-13_1750_01/settings.isf',
 'kwd': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/streams.kwd',
 'kwik': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/spikes.kwik',
 'kwe': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/events.kwe',
 'mda_raw': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-13_1750_01/raw.mda'}